In [ ]:
# Instale o Lark, se necessário
!pip install lark-parser --user

In [3]:
from pprint import pprint
from collections import ChainMap
from lark import Lark, InlineTransformer, Tree, Token

# Truque para fazer as árvores mostrarem no modo "pretty" por padrão
Tree._repr_html_ = lambda t: '<pre>%s</pre>' % t.pretty()

# Uma linguagem de programação simples
## Calculadora tradicional

Começamos nossa atividade criando uma calculadora tradicional que suporta as quatro operações básicas e mais a multiplicação. Usamos um método usual de definir a precedência e associatividade dos operadores a partir do modo como construímos a gramática.

In [4]:
calc = Lark(r"""
?start : block

?block : (item) (";" item)* ";"?

assign : NAME "=" expr

?item  : expr 
       | assign 
       | fn
       
?fn   : "fn" NAME "(" NAME ")" "{" block "}"

?expr : expr PLUS_OP term  -> binop
      | term
      
?term : expr MUL_OP term   -> binop
      | pow
      
?pow  : atom POW_OP pow    -> binop
      | atom
      
?atom : NUMBER -> number
      | NAME   -> name
      | "(" expr ")"
      | NAME "(" expr ")"  -> call


// Terminais
NUMBER  : /\d+(\.\d+)?/
NAME    : /[a-zA-Z]\w*/
POW_OP  : /\^/
PLUS_OP : /[+-]/
MUL_OP  : /[*\/]/
%ignore /\s+/

""")

Esta gramática consegue reconhecer facilmente expressões matemáticas simples

In [22]:
calc.parse('(123 + 1) + 3.14^2')

Tree(binop, [Tree(binop, [Tree(number, [Token(NUMBER, '123')]), Token(PLUS_OP, '+'), Tree(number, [Token(NUMBER, '1')])]), Token(PLUS_OP, '+'), Tree(binop, [Tree(number, [Token(NUMBER, '3.14')]), Token(POW_OP, '^'), Tree(number, [Token(NUMBER, '2')])])])

### Transformer

Vamos aplicar um transformer para converter as árvores geradas pelo Lark na representação de listas (também conhecidas como *S-expressions*, ou expressões simbólicas).

Note que existem poucas regras para considerar, já que temos apenas valores terminais (NUMBER e NAME) e expressões binárias nas nossas árvores.

In [6]:
class SExprTransformer(InlineTransformer):
    # Estre truque automaticamente cria um método que transforma o argumento
    # respectivamente para float e str
    number = float
    name = str
    
    # Passamos os elementos que formam a árvore como argumentos
    def binop(self, left, op, right):
        # convertemos operador de Token lark para string
        op = str(op)
        
        # Retorna a S-expression
        return (op, left, right)
    
    def assign(self, name, expr):
        return ('=', str(name), expr)
    
    def block(self, *args):
        return ('block', *args)
    
    def call(self, name, arg):
        return (str(name), arg)
    
    def fn(self, name, argname, block):
        return ('fn', str(name), str(argname), block)

    
tree = calc.parse('(123 + 1) + 3.14^2')
SExprTransformer().transform(tree)

('+', ('+', 123.0, 1.0), ('^', 3.14, 2.0))

### Avaliador

Agora que temos o parser e o transformer, vamos criar uma função que avalia o resultado de uma expressão matemática e retorna o valor associado. Tal função recebe uma árvore sintática na forma de uma S-expr e um dicionário de contexto que mapeia nomes a valores e retorna o resultado da expressão matemática correspondente.

In [39]:
def eval_sexpr(expr, ctx):
    # Se a expressão já é um número, não é necessário fazer nada    
    if isinstance(expr, (float, int)):
        return expr
    
    # Caso seja uma string, olhamos no dicinário de contexto
    elif isinstance(expr, str):
        return ctx[expr]
    
    # Finalmente, deve ser uma S-expression. Separamos o primeiro termo
    # dos argumentos e avaliamos o resultado condicionalmente
    head, *args = expr
    if head == '+':
        x, y = args
        return eval_sexpr(x, ctx) + eval_sexpr(y, ctx)
    elif head == '-':
        x, y = args
        return eval_sexpr(x, ctx) - eval_sexpr(y, ctx)
    elif head == '*':
        x, y = args
        return eval_sexpr(x, ctx) * eval_sexpr(y, ctx)
    elif head == '/':
        x, y = args
        return eval_sexpr(x, ctx) / eval_sexpr(y, ctx)
    elif head == '^':
        x, y = args
        return eval_sexpr(x, ctx) ** eval_sexpr(y, ctx)
    elif head == '=':
        name, value = args
        value = eval_sexpr(value, ctx)
        ctx[name] = value
        return value
    elif head == 'block':
        value = 0.0
        for arg in args:
            value = eval_sexpr(arg, ctx)
        return value
    elif head == 'fn':
        name, argname, block = args
        
        def fn(x):
            local_ctx = ChainMap({}, ctx)
            local_ctx[argname] = x
            return eval_sexpr(block, local_ctx)
        
        ctx[name] = fn
        return fn
    elif head in ctx:
        fn = ctx[head]
        return fn(*(eval_sexpr(x, ctx) for x in args))
    else:
        raise ValueError('argumento inválido para S-expression: %r' % head)
    

Testamos com um exemplo:

In [40]:
tree = calc.parse('(1 + 2) + 3 * x')
sexpr = SExprTransformer().transform(tree)
value = eval_sexpr(sexpr, {'x': 4})

print('Resultado:', value)

Resultado: 15.0


A série de operações 1) análise sintática (parsing); 2) transformação para *S-expression*; 3) avaliação da *S-expression* com um dicionário de contexto pode ser facilmente automatizada com uma função:

In [41]:
def calculate(expr, ctx=None):
    if ctx is None:
        ctx = {}
    tree = calc.parse(expr)
    sexpr = SExprTransformer().transform(tree)
    value = eval_sexpr(sexpr, ctx)
    return value

Testamos ...

In [42]:
calculate('x + 1', {'x': 41})

42.0

## Calculadora com passos

Vamos modificar a nossa calculadora para que ela aceite atribuição de variáveis e consiga calcular uma sequência de expressões. Deste modo, será possível criar pequenos programas como:

```
x = 40;
y = 2;
x + y
```

### Gramática

É possível reaproveitar boa parte da gramática utilizada anteriormente e acrescentar apeanas algumas regras para contemplar atribuição de variáveis e sequência de passos. Modifique a célula que define a gramática para contemplar as novas regras e rode os testes abaixo para verificar se foi implementado corretamente.

**OBS:** Os testes assumem que o nome da regra de atribuição é "assign" e da regra de geração de um bloco de instruções de um programa é "block". Lembre-se de modificar a regra inicial para procurar por um programa e não uma expressão.

In [43]:
calc.parse('''
x = 40;
y = 2;
x + y;
''')

Tree(block, [Tree(assign, [Token(NAME, 'x'), Tree(number, [Token(NUMBER, '40')])]), Tree(assign, [Token(NAME, 'y'), Tree(number, [Token(NUMBER, '2')])]), Tree(binop, [Tree(name, [Token(NAME, 'x')]), Token(PLUS_OP, '+'), Tree(name, [Token(NAME, 'y')])])])

In [44]:
# Teste para verificar se a as novas regras foram implementadas corretamente
src = "x = 1; x + 1"
expected = """
block
  assign
    x
    number	1
  binop
    name	x
    +
    number	1
"""
assert calc.parse(src).pretty() == expected.lstrip()
print('Parabéns!')

Parabéns!


### Transformer

As mudanças na gramática requerem algumas mudanças no transformer para lidar com as regras de "program" e "assign". Modifique a célula que define o SExprTransformer para incluir estas duas regras.

In [45]:
transformer = SExprTransformer()
transformer.transform(calc.parse('''
x = 40;
y = 2;
x + y
'''))

('block', ('=', 'x', 40.0), ('=', 'y', 2.0), ('+', 'x', 'y'))

In [46]:
# Teste para verificar se o transformer foi modificado corretamente
src = "x = 1; x + 1"
expected = (
    'block', 
    ('=', 'x', 1),
    ('+', 'x', 1),
)
transformer = SExprTransformer()
assert transformer.transform(calc.parse(src)) == expected
print('Parabéns!')

Parabéns!


### Função eval

Agora modificamos eval_sexpr para lidar com as duas novas regras. 

Podemos tratar a atribuição (regras do tipo "assign") simplesmente salvando o resultado do valor no lado direito a uma entrada no dicionário de contexto com o mesmo nome da variável do lado esquerdo. Isto pode ser feito alterando a estrutura de "if"s de eval_sexpr para aceitar regras que iniciam com um '='. Usamos a convenç~ao de assumir que o valor associado a um comando de atribuição corresponde à variável que acabou de ser salva.

Já para implementar as regras do tipo "block", devemos executar a sequência de operaões dentro de um laço, avaliando cada comando com uma chamada recursiva a "eval_expr". Aqui, usamos a convenção que o valor de uma sequência de comandos é igual ao valor do último comando. Como todos os valores na nossa linguagem são numéricos, podemos assumir um valor de 0.0 a qualquer sequência vazia.

Implemente as duas funcionalidades no códgo de "eval_expr".

In [47]:
ctx = {'y': 2}
value = eval_sexpr(('block', ('=', 'x', 1), ('+', 'x', 'y')), ctx)
print('ctx:', ctx)
print('resultado:', value)

ctx: {'y': 2, 'x': 1}
resultado: 3


In [48]:
# Teste para verificar se eval_sexpr foi modificado corretamente
assert calculate("x = 1; x + 1", {}) == 2
print('Parabéns!')

Parabéns!


## Chamada de funções

Vamos prosseguir de maneira similar à seção anterior para implementar uma calculadora que consiga chamar funções. A regra básica é que uma chamada de função possui a forma "func(arg)", onde func pode ser qualquer nome válido e arg pode ser qualquer expresssão válida. 

Uma expressão do tipo `sqrt(40 + 2)`, portanto, deve ser transformada para a *S-expression* ('sqrt', ('+', 40, 2)). Avaliamos esta expressão buscando o primeiro item da lista ('sqrt', neste caso) no dicionário de contexto, assumindo que o valor obtido se trata de uma função. Finalmente, usamos eval_sexpr para avaliar o argumento e chamamos a função com o resultado obtido para terminar a avaliação da chamada de função.

Note que para isto funcionar, devemos passar todas as funções reconhecidas na linguagem dentro do dicionário de contexto.

Implemente todos estes passos na gramática e verifique se funcionam nos testes abaixo.


In [49]:
import math

expr = 'x = 40; x + sqrt(4)'
tree = calc.parse(expr)
sexpr = SExprTransformer().transform(tree)

print('Árvore sintática:')
display(tree)
print('S-expr:', sexpr)
print('Resultado:', calculate(expr, {'sqrt': math.sqrt}))

Árvore sintática:


Tree(block, [Tree(assign, [Token(NAME, 'x'), Tree(number, [Token(NUMBER, '40')])]), Tree(binop, [Tree(name, [Token(NAME, 'x')]), Token(PLUS_OP, '+'), Tree(call, [Token(NAME, 'sqrt'), Tree(number, [Token(NUMBER, '4')])])])])

S-expr: ('block', ('=', 'x', 40.0), ('+', 'x', ('sqrt', 4.0)))
Resultado: 42.0


In [50]:
# Verifica se o resultado está correto

expr = 'x = 40; x + sqrt(4)'
tree = calc.parse(expr)
sexpr = SExprTransformer().transform(tree)

assert sexpr == ('block', ('=', 'x', 40.0), ('+', 'x', ('sqrt', 4.0)))
assert calculate(expr, {'sqrt': math.sqrt}) == 42.0
assert calculate(expr, {'sqrt': lambda x: x}) == 44.0
print('Parabéns!')

Parabéns!


## Definição de funções

Agora daremos um salto importante na nossa linguagem e permitiremos que os usuários definam as suas próprias funções. Uma função definida pelo usuário pode ser salva no dicionário de contexto e acessada normalmente se for invocada em um código posterior. A sintaxe proposta para isso é indicada abaixo:

```
fn double(x) {
    y = x;
    x + y
};

double(21)
```

(Lembre-se do ponto-vírgula depois da definição da função. É possível eliminá-lo, mas isso causaria mudanças relativamente grandes na gramática.)

Assim como nos passos anteriores, crie a regra gramatical e o transformer que suporta esta nova funcionalidade na gramática. Os testes abaixos supõe que uma definição de função seria representada como `('fn' <nome>, <nome-argumento> <corpo>)`.

**DICA:** lembre-se que Python permite criar funções dentro de funções. Desta forma, podemos criar a função a partir de uma expressão fn definindo uma função genérica associada ao bloco de comandos no corpo da função e associando o resultado ao seu nome no dicionário de contexto.

**DICA 2:** Não se preocupe em implementar contexto ainda. Assim, se uma função recebe um argumento x, devemos tratá-lo como global e salvar o valor de x no dicionário de contexto. Na nossa linguagem, tratamos todas variáveis como se fossem globais.

In [51]:
src = '''
fn double(x) {
    y = x;
    x + y
};

double(21)
'''

tree = calc.parse(src)
sexpr = SExprTransformer().transform(tree)

print('Árvore:\n', tree.pretty())
print('S-expr:'); pprint(sexpr)
print('Valor:', calculate(src))

Árvore:
 block
  fn
    double
    x
    block
      assign
        y
        name	x
      binop
        name	x
        +
        name	y
  call
    double
    number	21

S-expr:
('block',
 ('fn', 'double', 'x', ('block', ('=', 'y', 'x'), ('+', 'x', 'y'))),
 ('double', 21.0))
Valor: 42.0


In [52]:
# Testamos o código acima para verificar se a implementação é válida
src = "fn double(x) { 2 * x }; double(21)"
sexpr = SExprTransformer().transform(calc.parse(src))

ctx = {}
assert sexpr == ('block', ('fn', 'double', 'x', ('*', 2.0, 'x')), ('double', 21.0))
assert calculate(src, ctx) == 42
assert set(ctx) == {'x', 'double'} or set(ctx) == {'double'}, ctx
print('Parabéns!')

Parabéns!


## Controle do escopo

O código anterior possui um problema sério no controle de escopo. Como todas as variáveis são globais, argumentos de funções e variáveis definidas dentro do escopo de uma função afetam o contexto de execução fora da função. Isto faz com que códigos como o abaixo produzam resultados não esperados, já que o argumento x da função conflita com a variável x do escopo global. 

```
fn double(x) { x + x };
x = 20;
y = double(x + 1);
x + y
```

O código avalia para 63 e não 62, como seria esperado. O motivo para isso é que a chamada de função double(x) modifica globalmente o valor desta variável. Este comportamento é bastante confuso e difere de como a grande maioria das linguagens de programação funciona.

Para consertar este problema, é necessário criar uma hieraquia de contextos de execução de forma que quando uma função executa, ela altere apeans um escopo local, mas possa acessar as variáveis do escopo exterior para leitura, caso seja necessário. 

O Python possui uma implementação nativa de uma estrutura de dados ideal para isto: o ChainMap. Um ChainMap é formado por uma sequência de dicionários de forma que apenas o primeiro deles é escrito, mas caso um valor não seja encontrado no mesmo, ele busca em cada dicionário que forma a sequência. No caso abaixo,

```python
d1 = {'x': 1}
d2 = {'y': 2}
ctx = ChainMap(d1, d2)
```

ctx se comporta como um dicionário que modifica d1 caso façamos uma atribuição,

```python
ctx['z'] = 3
pŕint(d1) # --> {'x': 1, 'z': 3}
```
e busca qualquer chave desconhecida em d2, caso ela não esteja presente em d1.

```python
print(ctx['y']) # --> 2
```

Podemos utilizar o ChainMap para fazer o controle de escopo fazendo uma pequena alteração na parte que lida com definições de funções em eval_expr(). Basta embrulhar o contexto dentro de um ChainMap e passar o contexto local para as chamadas dentro da função.

In [53]:
ctx = {}
value = calculate('fn double(x) {x + x}; double(2)', ctx)
print('Resultado:', value)
print('Contexto:', ctx)

Resultado: 4.0
Contexto: {'double': <function eval_sexpr.<locals>.fn at 0x7f0b9ff54200>}


In [54]:
# Testamos o código acima para verificar se a implementação é válida
ctx = {}
src = "fn double(x) { 2 * x }; double(21)"
assert calculate(src, ctx) == 42
assert 'x' not in ctx, 'ainda está contaminando o escopo global!'
print('Parabéns!')

Parabéns!


## Outros desafios

Se você chegou até aqui, tente implementar mais algumas lacunas que estão faltando na nossa linguagem:

1. Implementar funções que suportam mais de um argumento.
2. Você consegue encurtar eval_sexpr() tratando os operadores +, -, *, etc como funções regulares?. Uma dica: utilize as funções add, sub, mul e truediv do módulo operator para simplificar mais ainda a sua implementação.
3. Suportar operadores de comparação e operadores lógicos. Operadores de comparação normalmente possuem uma precedência menor que os aritméticos, mas maior que operadores lógicos como (and, or, not). Escolha como será a gramática da sua linguagem.
4. Tente eliminar o ponto-vírgula obrigatório do fim das expressões. 
5. Implementar condicionais usando "if/elif/else" ou a sintaxe que achar mais interessante.
6. Implemente laços (while/for).